# 新版資料庫

搭啷！這個就是我們新版的資料庫，新增了
* 殖利率
* 三大法人
* 上櫃所有股票
* 更快速的存取方式

In [1]:
from finlab.crawler import (

    # 爬蟲們
    crawl_price,
    crawl_bargin,
    crawl_pe,
    crawl_monthly_report,
    crawl_finance_statement_by_date,
    crawl_twse_divide_ratio,
    crawl_otc_divide_ratio,
    crawl_twse_cap_reduction,
    crawl_otc_cap_reduction,
    crawl_benchmark,
    
    # 時間的range
    date_range,
    month_range,
    season_range,
    
    # GUI介面
    widget, out,
)

widget('price_2021', crawl_price, date_range)
#widget('price', crawl_price, date_range)
#widget('bargin_report', crawl_bargin, date_range)
widget('bargin_report_new', crawl_bargin, date_range)
widget('pe', crawl_pe, date_range)
#widget('monthly_report', crawl_monthly_report, month_range)
widget('monthly_report_new', crawl_monthly_report, month_range)
widget('financial_statement', crawl_finance_statement_by_date, season_range)
widget('twse_divide_ratio', crawl_twse_divide_ratio)
widget('otc_divide_ratio', crawl_otc_divide_ratio)
widget('twse_cap_reduction', crawl_twse_cap_reduction)
widget('otc_cap_reduction', crawl_otc_cap_reduction)

out.clear_output()
display(out)

Output(layout=Layout(border='1px solid black'))

# AUTO update 每日自動執行更新

有時候你可能會希望將所有的程式

每天自動執行就好，不然還要開jupyter -> 啟動notebook -> 執行GUI -> 點擊好幾次GUI

才能完成更新的動作
.
接下來我們就來教怎麼樣直接一次更新吧！

In [2]:
from finlab.crawler import table_date_range, update_table, to_pickle, commit
import datetime
from inspect import signature
from finlab.crawler import (

    # 爬蟲們
    crawl_price,
    crawl_bargin,
    crawl_pe,
    crawl_monthly_report,
    crawl_finance_statement_by_date,
    crawl_twse_divide_ratio,
    crawl_otc_divide_ratio,
    crawl_twse_cap_reduction,
    crawl_otc_cap_reduction,
    crawl_benchmark,
    
    # 時間的range
    date_range,
    month_range,
    season_range,
    
    # GUI介面
    widget, out,
)

def auto_update(table_name, crawl_function, time_range=None):

    sig = signature(crawl_function)
    print(table_name)

    if len(sig.parameters) != 0:
        first_date, last_date = table_date_range(table_name)
        dates = time_range(last_date, datetime.datetime.now())
        if dates:
            update_table(table_name, crawl_function, dates)
    else:
        df = crawl_function()
        to_pickle(df, table_name)
        


        

auto_update('price_2021', crawl_price, date_range)
auto_update('bargin_report_new', crawl_bargin, date_range)
auto_update('pe', crawl_pe, date_range)
auto_update('monthly_report_new', crawl_monthly_report, month_range)
auto_update('financial_statement', crawl_finance_statement_by_date, season_range)
auto_update('twse_divide_ratio', crawl_twse_divide_ratio)
auto_update('otc_divide_ratio', crawl_otc_divide_ratio)
auto_update('twse_cap_reduction', crawl_twse_cap_reduction)
auto_update('otc_cap_reduction', crawl_otc_cap_reduction)

commit()

price_2021
bargin_report_new
pe
monthly_report_new
financial_statement
twse_divide_ratio
twse_divide_ratio -- 資料日期
twse_divide_ratio -- 股票代號
twse_divide_ratio -- 股票名稱
twse_divide_ratio -- 除權息前收盤價
twse_divide_ratio -- 除權息參考價
twse_divide_ratio -- 權值+息值
twse_divide_ratio -- 權/息
twse_divide_ratio -- 漲停價格
twse_divide_ratio -- 跌停價格
twse_divide_ratio -- 開盤競價基準
twse_divide_ratio -- 減除股利參考價
twse_divide_ratio -- 詳細資料
twse_divide_ratio -- 最近一次申報資料 季別/日期
twse_divide_ratio -- 最近一次申報每股 (單位)淨值
twse_divide_ratio -- 最近一次申報每股 (單位)盈餘
twse_divide_ratio -- twse_divide_ratio
otc_divide_ratio
otc_divide_ratio -- 除權息日期
otc_divide_ratio -- 代號
otc_divide_ratio -- 名稱
otc_divide_ratio -- 除權息前收盤價
otc_divide_ratio -- 除權息參考價
otc_divide_ratio -- 權值
otc_divide_ratio -- 息值
otc_divide_ratio -- 權+息值
otc_divide_ratio -- 權/息
otc_divide_ratio -- 漲停價格
otc_divide_ratio -- 跌停價格
otc_divide_ratio -- 開盤競價基準
otc_divide_ratio -- 減除股利參考價
otc_divide_ratio -- 現金股利
otc_divide_ratio -- 每千股無償配股
otc_divide_ratio -- -
otc_divide_ratio -- 現

=== 移除多餘的 購售 7 開頭編碼 ===

In [8]:
from finlab.crawler import table_date_range, update_table, to_pickle, out, commit,merge, date_range
import datetime
from io import StringIO
import requests
from bs4 import BeautifulSoup
import time

import numpy as np
#import matplotlib.pyplot as plt
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
#import pandas_datareader.data as web

import os
from openpyxl import load_workbook


df = pd.read_pickle("history/tables/price_2021.pkl")
#print(len(df.index.get_level_values('stock_id').drop_duplicates()))
orgList=df.index.get_level_values('stock_id').drop_duplicates()
#print(df.shape)

toRemove=[]
for i in range(0,len(orgList)):
    if orgList[i][0:1] == '7':    #認購權證
        toRemove.append(orgList[i])
    if orgList[i][-1:] == '特' and len(orgList[i].split()[0]) > 4:  #特別股
        toRemove.append(orgList[i])

print('before cleaning up',len(orgList))
print('to clean up',len(toRemove))
df_cleaned=df.drop(index=toRemove,level=0)
print('after cleaning up',len(df_cleaned.index.get_level_values('stock_id').drop_duplicates()))
cleanedList=df_cleaned.index.get_level_values('stock_id').drop_duplicates()
for i in range(0,len(cleanedList)):
    print(i, cleanedList[i])

before cleaning up 12041
to clean up 9787
after cleaning up 2254
0 0015 富邦
1 0050 元大台灣50
2 0051 元大中型100
3 0052 富邦科技
4 0053 元大電子
5 0054 元大台商50
6 0055 元大MSCI金融
7 0056 元大高股息
8 0057 富邦摩台
9 0058 富邦發達
10 0059 富邦金融
11 0060 新台灣
12 0061 元大寶滬深
13 006201 元大富櫃50
14 006203 元大MSCI台灣
15 006204 永豐臺灣加權
16 006205 富邦上証
17 006206 元大上證50
18 006207 FH滬深
19 006208 富邦台50
20 00625K 富邦上証+R
21 00631L 元大台灣50正2
22 00632R 元大台灣50反1
23 00633L 富邦上証正2
24 00634R 富邦上証反1
25 00635U 元大S&P黃金
26 00636 國泰中國A50
27 00636K 國泰中國A50+U
28 00637L 元大滬深300正2
29 00638R 元大滬深300反1
30 00639 富邦深100
31 00640L 富邦日本正2
32 00641R 富邦日本反1
33 00642U 元大S&P石油
34 00643 群益深証中小
35 00643K 群益深証中小+R
36 00645 富邦日本
37 00646 元大S&P500
38 00647L 元大S&P500正2
39 00648R 元大S&P500反1
40 00649 FH香港
41 00650L FH香港正2
42 00651R FH香港反1
43 00652 富邦印度
44 00653L 富邦印度正2
45 00654R 富邦印度反1
46 00655L 國泰中國A50正2
47 00656R 國泰中國A50反1
48 00657 國泰日經225
49 00657K 國泰日經225+U
50 00658L 國泰日本正2
51 00659R 國泰日本反1
52 00660 元大歐洲50
53 00661 元大日經225
54 00662 富邦NASDAQ
55 00663L 國泰臺灣加權正2
56 00664R 國泰

## 加入新的benchmark data

In [46]:
# 時間物件
import datetime
import requests
import pandas as pd
from io import StringIO


# 下載某天的資料

date = datetime.date(2018,1,3)

def crawl_benchmark(date):
    date_str = date.strftime('%Y%m%d')
    res = requests.get("http://www.twse.com.tw/exchangeReport/MI_5MINS_INDEX?response=csv&date=" + 
                       date_str + "&_=1544020420045")
    
    if len(res.text) < 100:
        print('holiday')
        return pd.DataFrame()

    # 利用 pandas 將資料整理成表格
    df = pd.read_csv(StringIO(res.text.replace("=","")), header=1, index_col='時間')

    # 資料處理
    df = df.dropna(how='all', axis=0).dropna(how='all', axis=1)
    df.index = pd.to_datetime(date.strftime('%Y %m %d ') + pd.Series(df.index))
    df = df.apply(lambda s: s.astype(str).str.replace(",", "").astype(float))

    df.head()

    df.index.name = 'date'
    df = df.reset_index()
    df['stock_id'] = '台股指數'
    df = df.set_index(['stock_id', 'date'])
    return df

from finlab.crawler import widget, date_range, out

widget('benchmark', crawl_benchmark, date_range)
#證交所資料  最早到 2004/10/15

out.clear_output()
display(out)
#commit()

Output(layout=Layout(border='1px solid black'))



## 1. 資料調用

In [39]:
import pandas as pd
df = pd.read_pickle("history/tables/benchmark.pkl")
#df1 = pd.read_pickle("history/tables/TWII_complete.pkl")
print('2004-10-15 09:00:00' in df.index.get_level_values('date'))
#print(df.tail())
#print(df1.head())
#print(df1.tail())
#print(df.shape)


True


In [43]:
print('2006-03-16 09:00:00' in df.index.get_level_values('date'))

True


## 2. 分解上述的功能

### 2.A 分解其中的功能 crawl_function

In [ ]:
import datetime
df = crawl_price(datetime.date(2019,1,2))
df.head()

### 2.B 分解其中功能 time_range

In [ ]:
season_range(datetime.date(2018,1,2), datetime.date(2019,1,10))

### 2.C 將功能組裝起來 widget

In [ ]:
widget("price", crawl_price, date_range)

## 3. 做一些小修改，方便我們之後做財經資料研究

In [28]:
from finlab.crawler import commit

commit()

已經成功commit過 balance_sheet 了，跳過！
已經成功commit過 bargin_report 了，跳過！
已經成功commit過 benchmark 了，跳過！
已經成功commit過 cash_flows 了，跳過！
已經成功commit過 income_sheet 了，跳過！
已經成功commit過 income_sheet_cumulate 了，跳過！
已經成功commit過 monthly_report_new 了，跳過！
已經成功commit過 otc_cap_reduction 了，跳過！
已經成功commit過 otc_divide_ratio 了，跳過！
已經成功commit過 pe 了，跳過！
已經成功commit過 price_new 了，跳過！
已經成功commit過 TWII_complete 了，跳過！
已經成功commit過 twse_cap_reduction 了，跳過！
已經成功commit過 twse_divide_ratio 了，跳過！


## 4. 將做完修改的資料拿出來

In [ ]:
df = pd.read_pickle("history/items/price/收盤價.pkl")
df.head()

## 查看存放好的benchmark data

In [13]:
from finlab.crawler import commit

commit()

已經成功commit過 balance_sheet 了，跳過！
已經成功commit過 bargin_report 了，跳過！
已經成功commit過 benchmark 了，跳過！
已經成功commit過 cash_flows 了，跳過！
已經成功commit過 income_sheet 了，跳過！
已經成功commit過 income_sheet_cumulate 了，跳過！
已經成功commit過 monthly_report_new 了，跳過！
已經成功commit過 otc_cap_reduction 了，跳過！
已經成功commit過 otc_divide_ratio 了，跳過！
已經成功commit過 pe 了，跳過！
已經成功commit過 price_new 了，跳過！
已經成功commit過 TWII_complete 了，跳過！
已經成功commit過 twse_cap_reduction 了，跳過！
已經成功commit過 twse_divide_ratio 了，跳過！


In [21]:
from finlab.data import Data

data = Data()

df = data.get("發行量加權股價指數")
df.head(50)
df.dropna()
df.head(50)

stock_id,台股指數
date,
2005-12-15 09:00:00,NaN
2005-12-15 09:01:00,NaN
2005-12-15 09:02:00,NaN
2005-12-15 09:03:00,NaN
2005-12-15 09:04:00,NaN
2005-12-15 09:05:00,NaN
2005-12-15 09:06:00,NaN
2005-12-15 09:07:00,NaN
2005-12-15 09:08:00,NaN


In [16]:
df.tail()

stock_id,台股指數
date,
2020-04-01 13:29:40,9686.02
2020-04-01 13:29:45,9686.02
2020-04-01 13:29:50,9686.02
2020-04-01 13:29:55,9686.02
2020-04-01 13:30:00,9663.63


from finlab.data import Data

data = Data()

df = data.get("Close")
print(df.head())
print(df.tail())

##自建台灣加權股價指數1997/1/1 ~

